In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load the data into a pandas DataFrame
data = pd.read_csv('retail_data.csv')

# Add a column to the DataFrame indicating whether each product is in stock or not
data['in_stock'] = data['productName'].isin(['Product1', 'Product2', 'Product3'])

# Create a pivot table of the ratings data
ratings_matrix = data.pivot_table(index='customerId', columns='productName', values='quantity')

# Compute the cosine similarity between each pair of users
user_similarity = cosine_similarity(ratings_matrix)

# Create a dictionary to store the recommendations for each user
recommendations = {}

# For each user, find the most similar users and get their product recommendations
for user in ratings_matrix.index:
    # Get the index of the user in the similarity matrix
    user_index = ratings_matrix.index.get_loc(user)

    # Get the indices of the most similar users
    similar_user_indices = user_similarity[user_index].argsort()[::-1][1:6]

    # Get the product ratings for the most similar users
    similar_user_ratings = ratings_matrix.iloc[similar_user_indices]

    # Get the average ratings for each product across the most similar users, excluding the user's own ratings and products that are not in stock
    product_ratings = similar_user_ratings.mean(axis=0) - ratings_matrix.loc[user]
    product_ratings = product_ratings[data['in_stock']]

    # Get the top 5 products with the highest average ratings
    top_products = product_ratings.sort_values(ascending=False)[:5]

    # Store the recommendations for the user
    recommendations[user] = list(top_products.index)

# Create a DataFrame with the recommendations for each user
recommendations_df = pd.DataFrame.from_dict(recommendations, orient='index')
recommendations_df.columns = ['recommended_product1', 'recommended_product2', 'recommended_product3', 'recommended_product4', 'recommended_product5']
